<a href="https://colab.research.google.com/github/kimestelle/CIS5810-video-analysis/blob/main/CIS581_G9_emot%2Btranscript_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall -y moviepy
!pip install moviepy==1.0.3

Found existing installation: moviepy 1.0.3
Uninstalling moviepy-1.0.3:
  Successfully uninstalled moviepy-1.0.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110712 sha256=9862b4960651eee534b3ab16c044119dfe5b9ad91640aa04c74c969b31865dea
  Stored in directory: /root/.cache/pip/wheels/df/ba/4b/0917fc0c8833c8ba7016565fc975b74c67bc8610806e930272
Successfully built moviepy


In [2]:
!pip install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install git+https://github.com/guillaumekln/faster-whisper.git moviepy deepface tqdm

Looking in indexes: https://download.pytorch.org/whl/cu121
  Cloning https://github.com/guillaumekln/faster-whisper.git to /tmp/pip-req-build-ce87atit
  Running command git clone --filter=blob:none --quiet https://github.com/guillaumekln/faster-whisper.git /tmp/pip-req-build-ce87atit
  Resolved https://github.com/guillaumekln/faster-whisper.git to commit 65882eee9f5cdbeeb2d877f1131d48cf241b327d
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 101.8 MB/s eta 0:00:00
   ━━━━━

In [4]:
# ==============================================
# 📦 Imports
# ==============================================
from faster_whisper import WhisperModel
import moviepy.editor as mp
from deepface import DeepFace
import numpy as np
import json
import cv2
import os
from tqdm import tqdm

# ==============================================
# 🎥 분석할 비디오 경로
# ==============================================
VIDEO_PATH = "kim.mp4"

# ==============================================
# 🎧 Step 1. Whisper (로컬)로 음성 → 텍스트 변환
# ==============================================
def transcribe_video(video_path, model_size="small"):
    print("🎧 Extracting audio and transcribing with local Whisper model...")

    clip = mp.VideoFileClip(video_path)
    audio_path = "temp_audio.mp3"
    clip.audio.write_audiofile(audio_path, codec="mp3", verbose=False, logger=None)

    device = "cuda" if os.system("nvidia-smi > /dev/null 2>&1") == 0 else "cpu"
    model = WhisperModel(model_size, device=device)

    segments, _ = model.transcribe(audio_path)
    transcript = [{"start": s.start, "end": s.end, "text": s.text} for s in segments]
    full_text = " ".join([seg["text"] for seg in transcript]).strip()

    with open("transcript.json", "w", encoding="utf-8") as f:
        json.dump(transcript, f, indent=2, ensure_ascii=False)

    print("✅ Transcription complete. Saved to transcript.json")
    return full_text, transcript

# ==============================================
# 😊 Step 2. DeepFace로 감정 분석 (로컬)
# ==============================================
def analyze_emotions(video_path, sample_rate=1.0):
    print("😊 Analyzing emotions with DeepFace...")

    clip = mp.VideoFileClip(video_path)
    duration = clip.duration
    times = np.arange(0, duration, sample_rate)

    emotions = []
    for t in tqdm(times, desc="Emotion detection"):
        frame = clip.get_frame(t)

        # 프레임 리사이징: 최대 800px
        h, w, _ = frame.shape
        scale = 800 / max(h, w) if max(h, w) > 800 else 1.0
        frame_resized = cv2.resize(frame, (int(w*scale), int(h*scale)))

        try:
            results = DeepFace.analyze(
                frame_resized,
                actions=['emotion'],
                enforce_detection=False,
                detector_backend='retinaface'
            )

            if not isinstance(results, list):
                results = [results]

            if len(results) > 0:
                for face_result in results:
                    # numpy.float32 → float 변환
                    scores = {k: float(v) for k, v in face_result.get("emotion", {}).items()}
                    emotions.append({
                        "time": float(t),
                        "dominant_emotion": face_result.get("dominant_emotion", "unknown"),
                        "emotion_scores": scores,
                        "num_faces": len(results)
                    })
            else:
                emotions.append({
                    "time": float(t),
                    "dominant_emotion": "no_face_detected",
                    "emotion_scores": {},
                    "num_faces": 0
                })

        except Exception as e:
            emotions.append({
                "time": float(t),
                "dominant_emotion": "error",
                "emotion_scores": {},
                "num_faces": 0,
                "error": str(e)
            })

    with open("emotion_analysis.json", "w", encoding="utf-8") as f:
        json.dump(emotions, f, indent=2, ensure_ascii=False)

    print("💾 Saved emotion data to emotion_analysis.json")
    return emotions


# ==============================================
# 🔗 Step 3. 텍스트 + 감정 데이터 병합
# ==============================================
def merge_data(transcript_text, emotions):
    print("🔗 Merging transcript and emotion data...")

    sentences = [s.strip() for s in transcript_text.split(". ") if s.strip()]
    merged = []

    for i, line in enumerate(sentences):
        merged.append({
            "text": line,
            "emotion": emotions[i]["dominant_emotion"] if i < len(emotions) else "neutral",
            "emotion_scores": emotions[i]["emotion_scores"] if i < len(emotions) else {},
            "time": emotions[i]["time"] if i < len(emotions) else None
        })

    with open("merged_analysis.json", "w", encoding="utf-8") as f:
        json.dump(merged, f, indent=2, ensure_ascii=False)

    print("💾 Saved merged data to merged_analysis.json")
    return merged

# ==============================================
# 🚀 전체 파이프라인 실행
# ==============================================
if __name__ == "__main__":
    print("🚀 Starting enhanced video analysis pipeline...\n")
    transcript_text, transcript_segments = transcribe_video(VIDEO_PATH)
    emotions = analyze_emotions(VIDEO_PATH, sample_rate=1.0)  # 1초 단위 샘플링
    merged = merge_data(transcript_text, emotions)
    print("\n✅ 전체 파이프라인 완료!")
    print(" - transcript.json")
    print(" - emotion_analysis.json")
    print(" - merged_analysis.json")


🚀 Starting enhanced video analysis pipeline...

🎧 Extracting audio and transcribing with local Whisper model...
✅ Transcription complete. Saved to transcript.json
😊 Analyzing emotions with DeepFace...


Emotion detection: 100%|██████████| 84/84 [12:52<00:00,  9.20s/it]

💾 Saved emotion data to emotion_analysis.json
🔗 Merging transcript and emotion data...
💾 Saved merged data to merged_analysis.json

✅ 전체 파이프라인 완료!
 - transcript.json
 - emotion_analysis.json
 - merged_analysis.json


In [5]:
import json

# 읽고 싶은 파일 목록
files = ["transcript.json", "emotion_analysis.json", "merged_analysis.json"]

for file in files:
    print(f"\n📂 --- {file} 내용 ---")
    try:
        with open(file, "r") as f:
            data = json.load(f)
            # JSON 내용 깔끔하게 출력
            print(json.dumps(data, indent=2, ensure_ascii=False))
    except Exception as e:
        print(f"⚠️ {file} 읽는 중 오류 발생:", e)



📂 --- transcript.json 내용 ---
[
  {
    "start": 0.0,
    "end": 2.0,
    "text": " What is with the merchandise?"
  },
  {
    "start": 2.0,
    "end": 6.0,
    "text": " Other day, a little kid in a store causing problems, crushing cheap."
  },
  {
    "start": 6.0,
    "end": 10.0,
    "text": " At my restaurant, they come in like it is their own living room."
  },
  {
    "start": 10.0,
    "end": 13.0,
    "text": " Wearing capes, spider-man pajamas,"
  },
  {
    "start": 13.0,
    "end": 17.0,
    "text": " tripping waiters, throwing chapatis like they are frisbees."
  },
  {
    "start": 17.0,
    "end": 20.0,
    "text": " And the parents do nothing."
  },
  {
    "start": 20.0,
    "end": 23.0,
    "text": " When I was a child, I could not even speak in public."
  },
  {
    "start": 23.0,
    "end": 26.0,
    "text": " Yeah. If I even look at my father,"
  },
  {
    "start": 26.0,
    "end": 29.0,
    "text": " but if I forgot to touch my grandmother's feet,"
  },
  {
    "